## Introduction

## Importing Necessary Libraries

In [2]:
!pip install matplotlib

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from IPython.display import Image

import ipywidgets as widgets
from IPython.display import display, clear_output

from math import * 
from dataclasses import dataclass, asdict
from enum import Enum
from typing import List, Tuple
import random
import pandas as pd
import time


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


## Defining Constants and Variables

In [35]:
class Loss(Enum):
    e_d = 0.1       # efficiency with detectors
    e_pbs1 = 0.7    # efficiency with pbs 1
    e_pbs2 = 0.7    # efficiency with pbs 2
    e_bs50 = 0.7    # efficiency with 50/50 beam splitter
    e_fff = 0.7     # effiency with fiber -> free space -> fiber
    
    e_hwp_a = 1  # efficiency with hwp a
    e_qwp_a = 1  # efficiency with qwp a
    e_hwp_b = 1   # efficiency with hwp b
    e_qwp_b = 1   # efficiency with qwp b

class Basis(Enum):
    Rectilinear = ("H", "V")
    Diagonal = ("D", "A")

class States(Enum):
    H = (1, 0)
    V = (0, 1)
    D = (1/1/sqrt(2), 1/1/sqrt(2))
    A = (1/1/sqrt(2), -1/1/sqrt(2))
    
    @property
    def basis(self):
        return Basis.Rectilinear if self.name in Basis.Rectilinear.value else Basis.Diagonal
    
    @property
    def bit(self):
        return 0 if self.name in ("H", "D") else 1
    
    def loss_value(self, loss_multiplier: float) -> Tuple[float]:
        return (self.value[0] * loss_multiplier, self.value[1] * loss_multiplier)

@dataclass
class Measurement:
    d1h: bool
    d1v: bool
    d2h: bool
    d2v: bool
    
    @property
    def bell_plus(self):
        return (self.d1h & self.d1v) | (self.d2h & self.d2v)
    
    @property
    def bell_minus(self):
        return (self.d1h & self.d2v) | (self.d1v & self.d2h)
    
@dataclass
class SimulationResult:
    alice_state: str
    alice_bit: int
    bob_state: str
    bob_bit: int
    
    basis: str
    bell_plus: bool
    bell_minus: bool
    is_valid: bool

## Defining Helper Functions

#### Expected Photon Count

In [36]:
N1v = lambda ah, av, bh, bv: (Loss.e_bs50.value * Loss.e_pbs1.value)**2 * (av - bv)**2 / 2
N1h = lambda ah, av, bh, bv: (Loss.e_bs50.value * Loss.e_pbs1.value)**2 * (ah - bh)**2 / 2
N2v = lambda ah, av, bh, bv: (Loss.e_bs50.value * Loss.e_pbs2.value)**2 * (av + bv)**2 / 2
N2h = lambda ah, av, bh, bv: (Loss.e_bs50.value * Loss.e_pbs2.value)**2 * (ah + bh)**2 / 2

#### Simulation

In [60]:
def simulate_measurement(alice: States, bob: States) -> Measurement:
    
    alice_loss = Loss.e_fff.value * Loss.e_hwp_a.value * Loss.e_qwp_a.value
    bob_loss = Loss.e_fff.value * Loss.e_hwp_b.value * Loss.e_qwp_b.value
    
    detector_weights = [
        Loss.e_d.value * N1v(*alice.loss_value(alice_loss), *bob.loss_value(bob_loss)),
        Loss.e_d.value * N1h(*alice.loss_value(alice_loss), *bob.loss_value(bob_loss)),
        Loss.e_d.value * N2v(*alice.loss_value(alice_loss), *bob.loss_value(bob_loss)),
        Loss.e_d.value * N2h(*alice.loss_value(alice_loss), *bob.loss_value(bob_loss)),
    ]
    
    scale_factor = 2 / sum(detector_weights)
    scaled_detector_weights = [scale_factor * weight for weight in detector_weights]
    
    detector_hits = random.choices(["d1v", "d1h", "d2v", "d2h"], weights=scaled_detector_weights, k=2)

    is_valid = random.random() <= 1/scale_factor
    d1v = "d1v" in detector_hits if is_valid else False
    d1h = "d1h" in detector_hits if is_valid else False
    d2v = "d2v" in detector_hits if is_valid else False
    d2h = "d2h" in detector_hits if is_valid else False

    return Measurement(d1h=d1h, d1v=d1v, d2h=d2h, d2v=d2v)


def simulate_experiment(steps: int) -> List[SimulationResult]:
    
    results = []
    
    for step in range(steps):
        alice = random.choice(list(States))
        bob = random.choice(list(States))
        measurement = simulate_measurement(alice, bob)
        is_same_basis = alice.basis == bob.basis
        basis = alice.basis.name if is_same_basis else "n/a"
        
        is_valid = (measurement.bell_plus | measurement.bell_minus) & is_same_basis
        
        
        simulation_result = SimulationResult(
            alice_state=alice.name, 
            alice_bit=alice.bit, 
            bob_state=bob.name, 
            bob_bit=bob.bit, 
            basis=basis, 
            bell_plus=measurement.bell_plus, 
            bell_minus=measurement.bell_minus,
            is_valid=is_valid,
        )
        results.append(simulation_result)
        
    return results


def apply_bit_flip(results_df: pd.DataFrame) -> pd.DataFrame:
    
    if results_df["basis"] == Basis.Rectilinear.name:
        return (results_df["bob_bit"] + 1) % 2
    
    if results_df["basis"] == Basis.Diagonal.name and results_df["bell_minus"]:
        return (results_df["bob_bit"] + 1) % 2

    return results_df["bob_bit"]



def calculate_key_rate():
    
    print("Calculating key rate...")
    
    key_rates = []

    start = time.time()
    for step in range(100, 10000, 100):

        results = simulate_experiment(step)

        df = pd.DataFrame(results)
        df = df[df["is_valid"]]

        # print(step, df.shape[0], round(df.shape[0] / step, 4))

        key_rates.append(df.shape[0] / step)

    end = time.time()

    avg_key_rate = round(np.mean(key_rates), 4) 
    std_key_rate = round(np.std(key_rates), 4)
    
    print(f"{avg_key_rate=}")
    print(f"{std_key_rate=}")
    print(f"Time to run: {round(end-start, 2)} seconds")
    print("----------------------------------")
    
    return avg_key_rate


def generate_key(target_key_size: int):
    
    key_rate = calculate_key_rate()
    multiplier = 1 / key_rate    
    steps = int(target_key_size * multiplier)
    print(f"Generating {target_key_size=} by simulating {steps=} with {key_rate=} \n")
    
    results = simulate_experiment(steps)
    df = pd.DataFrame(results)
    df = df[df["is_valid"]]
    df["bob_bit_sifted"] = df.apply(apply_bit_flip, axis=1)
    
    alice_key = ''.join(df['alice_bit'].astype(str))
    bob_key = ''.join(df['bob_bit_sifted'].astype(str))
    
    alice_key_hex = hex(int(alice_key, 2))
    bob_key_hex = hex(int(bob_key, 2))

    assert alice_key == bob_key, "Keys do not match"
    
    return alice_key, bob_key, alice_key_hex, bob_key_hex


def calculate_error_rate(str1, str2):
    if len(str1) != len(str2):
        raise ValueError("Strings must have the same length")

    differences = sum(ch1 != ch2 for ch1, ch2 in zip(str1, str2))
    error_rate = differences / len(str1)
    return error_rate

## Simulating Experiment

In [61]:
results = simulate_experiment(1000)

df = pd.DataFrame(results)
df = df[df["is_valid"]]

df

,alice_state,alice_bit,bob_state,bob_bit,basis,bell_plus,bell_minus,is_valid
33,A,1,A,1,Diagonal,True,False,True
363,D,0,D,0,Diagonal,True,False,True
890,H,0,V,1,Rectilinear,False,True,True


In [ ]:
desired_key_size = 100
alice_key, bob_key, alice_key_hex, bob_key_hex = generate_key(desired_key_size)
key_size = len(alice_key)
error_rate = calculate_error_rate(alice_key, bob_key)


print(f"{alice_key=}")
print(f"{bob_key=}")

print(f"{alice_key_hex=}")
print(f"{bob_key_hex=}")

print(f"{key_size=}")
print(f"{error_rate=}")

Calculating key rate...


### Testing

In [ ]:
for alice in States:
    for bob in States:
        
        same_basis = alice.basis == bob.basis
        
        print(f"{alice.name}, {bob.name}: {same_basis}")

In [ ]:
steps = 100000


outputs = {
    "Alice": [],
    "Bob": [],
    "d1v": [],
    "d1h": [],
    "d2v": [],
    "d2h": []
}

for alice in States:
    for bob in States:
        
        d1 = 0
        d2 = 0
        d3 = 0
        d4 = 0
        
        for i in range(steps):
            detector_weights = [
                N1v(*alice.value, *bob.value),
                N1h(*alice.value, *bob.value),
                N2v(*alice.value, *bob.value),
                N2h(*alice.value, *bob.value),
            ]
    
            detector_hits = random.choices(["d1v", "d1h", "d2v", "d2h"], weights=detector_weights, k=2)
            d1 += detector_hits.count("d1v")
            d2 += detector_hits.count("d1h")
            d3 += detector_hits.count("d2v")
            d4 += detector_hits.count("d2h")

        
        outputs["Alice"].append(alice)
        outputs["Bob"].append(bob)


        outputs['d1v'].append(round(d1/steps, 2))
        outputs['d1h'].append(round(d2/steps, 2))
        outputs['d2v'].append(round(d3/steps, 2))
        outputs['d2h'].append(round(d4/steps, 2))
        
df = pd.DataFrame(outputs)  
df = df.set_index(["Alice", "Bob"])  

df

In [ ]:
steps = 10000


outputs = {
    "Alice": [],
    "Bob": [],
    "d1v": [],
    "d1h": [],
    "d2v": [],
    "d2h": [],
    "hit_prob": []
}

for alice in States:
    for bob in States:
        
        d1 = 0
        d2 = 0
        d3 = 0
        d4 = 0
        
        alice_loss = Loss.e_fff.value * Loss.e_hwp_a.value * Loss.e_qwp_a.value
        bob_loss = Loss.e_fff.value * Loss.e_hwp_b.value * Loss.e_qwp_b.value

        detector_weights = [
            Loss.e_d.value * N1v(*alice.loss_value(alice_loss), *bob.loss_value(bob_loss)),
            Loss.e_d.value * N1h(*alice.loss_value(alice_loss), *bob.loss_value(bob_loss)),
            Loss.e_d.value * N2v(*alice.loss_value(alice_loss), *bob.loss_value(bob_loss)),
            Loss.e_d.value * N2h(*alice.loss_value(alice_loss), *bob.loss_value(bob_loss)),
        ]

        scale_factor = 2 / sum(detector_weights)
        scaled_detector_weights = [scale_factor * weight for weight in detector_weights]
        
        
        for i in range(steps):
            
            detector_hits = random.choices(["d1v", "d1h", "d2v", "d2h"], weights=scaled_detector_weights, k=2)

            is_valid = random.random() <= 1/scale_factor
            d1v += detector_hits.count("d1v") if is_valid else 0
            d1h += detector_hits.count("d1h") if is_valid else 0
            d2v += detector_hits.count("d2v") if is_valid else 0
            d2h += detector_hits.count("d2h") if is_valid else 0

        
        outputs["Alice"].append(alice)
        outputs["Bob"].append(bob)


        outputs['d1v'].append(round(d1v/steps, 5))
        outputs['d1h'].append(round(d1h/steps, 5))
        outputs['d2v'].append(round(d2v/steps, 5))
        outputs['d2h'].append(round(d2h/steps, 5))
        outputs['hit_prob'].append(round(1/scale_factor, 5))
        
        
df = pd.DataFrame(outputs)  
df = df.set_index(["Alice", "Bob"])  

df

In [49]:
key_rates = []

start = time.time()
for step in range(100, 10000, 100):
    
    results = simulate_experiment(step)

    df = pd.DataFrame(results)
    df = df[df["is_valid"]]

    # print(step, df.shape[0], round(df.shape[0] / step, 5))
    
    key_rates.append(df.shape[0] / step)

end = time.time()

avg_key_rate = round(np.mean(key_rates), 5) 
std_key_rate = round(np.std(key_rates), 5)

print(f"{avg_key_rate=}")
print(f"{std_key_rate=}")
print(f"Time to run: {round(end-start, 2)} seconds")

avg_key_rate=0.0022
std_key_rate=0.001
Time to run: 26.05 seconds
